# Qwen-Image 多GPU文本生成图像测试

本笔记本使用多GPU协同工作来运行Qwen-Image模型，解决单GPU显存不足的问题。

## 硬件要求
- 2个或更多GPU（每个至少35GB显存）
- 总显存需求：约70GB

## 多GPU工作原理
- 每个GPU独立加载一个模型实例
- 通过任务队列分配工作负载
- 支持并发处理多个图像生成任务

In [ ]:
!pip install modelscope ipywidgets tqdm accelerate diffusers==0.35.1 transformers gradio

In [ ]:
from model_download import download_model

download_model()

In [ ]:
from importlib.metadata import version
print("diffusers version:", version("diffusers"))
print("torch version:", version("torch"))
print("transformers version:", version("transformers"))

In [ ]:
import torch
import os

# 检测GPU配置
if torch.cuda.is_available():
    torch_dtype = torch.bfloat16
    device = "cuda"
    num_gpus = torch.cuda.device_count()
    print(f"检测到 {num_gpus} 个GPU:")
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
        print(f"  GPU {i}: {gpu_name} ({gpu_memory:.1f}GB)")
else:
    torch_dtype = torch.float32
    device = "cpu"
    num_gpus = 0
    print("未检测到CUDA GPU，将使用CPU")

print(f"\n使用数据类型: {torch_dtype}")
print(f"主设备: {device}")

In [ ]:
# 配置多GPU环境变量
NUM_GPUS_TO_USE = min(2, num_gpus)  # 最多使用2个GPU
os.environ["NUM_GPUS_TO_USE"] = str(NUM_GPUS_TO_USE)
os.environ["TASK_QUEUE_SIZE"] = "10"
os.environ["TASK_TIMEOUT"] = "300"

print(f"配置使用 {NUM_GPUS_TO_USE} 个GPU")
print(f"任务队列大小: {os.environ['TASK_QUEUE_SIZE']}")
print(f"任务超时时间: {os.environ['TASK_TIMEOUT']}秒")

if NUM_GPUS_TO_USE < 1:
    raise RuntimeError("需要至少1个GPU来运行此笔记本")

In [ ]:
# 导入多GPU管理器
import sys
import time
sys.path.append('./src/examples')

try:
    from demo import MultiGPUManager
    print("成功导入多GPU管理器")
except ImportError as e:
    print(f"导入多GPU管理器失败: {e}")
    print("请确保 src/examples/demo.py 文件存在")
    raise

In [ ]:
# 初始化多GPU管理器
model_name = "./models/Qwen-Image"

print("正在初始化多GPU管理器...")
print("注意：这个过程可能需要几分钟，因为每个GPU都需要加载完整的模型")

gpu_manager = MultiGPUManager(
    model_repo_id=model_name,
    num_gpus=NUM_GPUS_TO_USE,
    task_queue_size=10
)

# 启动GPU工作进程
gpu_manager.start_workers()

# 等待所有GPU初始化完成
print("等待所有GPU初始化完成...")
time.sleep(10)  # 给GPU一些时间来初始化

print("多GPU管理器初始化完成！")

In [ ]:
# 设置测试参数
positive_magic = {
    "en": ", Ultra HD, 4K, cinematic composition.", # for english prompt
    "zh": ", 超清，4K，电影级构图." # for chinese prompt
}

# 定义5个测试提示词
test_prompts = [
    {
        "name": "cyberpunk_city",
        "prompt": '''A photo of a cyberpunk city at night, with neon lights reflecting on the rain-slicked streets. A ramen stall's sign clearly reads "未来拉面" (Future Ramen), with smaller text below saying "AI-Powered Noodles." A nearby electronic billboard scrolls with binary code: "01010100 01101111 01101110 01100111 01111001 01101001".''',
        "description": "测试霓虹灯效果下的中英文文本渲染，以及复杂背景下数字/代码的清晰度"
    },
    {
        "name": "ancient_book",
        "prompt": '''A close-up shot of a yellowed, ancient book open on a heavy wooden desk, with a quill pen beside it. On the page, beautiful calligraphy reads "靜以修身，儉以養德" (Cultivate oneself in tranquility, nurture virtue through thrift). At the bottom of the page, a faded English note says: "The virtues are cultivated in tranquility".''',
        "description": "测试不同中文书法风格和手写斜体英文的渲染，以及文本在羊皮纸等古老材质上的纹理"
    },
    {
        "name": "product_packaging",
        "prompt": '''A high-end, close-up shot of a modern, minimalist skincare product box. The front of the box features the product name "光子精华" (Photon Essence) and the brand "Quantum Glow." The ingredient list is clearly visible: "Aqua, Glycerin, Peptide-5, Hyaluronic Acid (玻尿酸)".''',
        "description": "测试现代产品设计上印刷文本的清晰度和纹理，评估模型处理中英混合排版、大小写敏感性和技术术语的能力"
    },
    {
        "name": "street_graffiti",
        "prompt": '''A weathered brick wall in a city alley, covered in colorful graffiti art. At the center of the graffiti is a large Chinese character "龙" (Dragon), with "DRAGON'S BREATH" written next to it in a spray-paint style font. In the corner, a small signature reads: "By Artist Qwen, 2024".''',
        "description": "测试非标准艺术字体（如喷漆和手写）的渲染，以及文本与砖墙粗糙纹理的融合效果"
    },
    {
        "name": "code_screen",
        "prompt": '''A programmer's desk with a computer monitor displaying code. A Python function is clearly visible on the screen: "def calculate_fibonacci(n): # 计算斐波那契数列\n    if n <= 1:\n        return n". A sticky note with the handwritten text "Launch Deadline: 12/25" is faintly visible in the screen's reflection.''',
        "description": "测试等宽字体（如代码字体）的准确渲染，包括缩进、语法高亮和双语注释，以及屏幕反射等复杂光学效果下的详细文本处理"
    }
]

negative_prompt = " " # Recommended if you don't use a negative prompt.

# Generate with different aspect ratios
aspect_ratios = {
    "1:1": (1328, 1328),
    "16:9": (1664, 928),
    "9:16": (928, 1664),
    "4:3": (1472, 1104),
    "3:4": (1104, 1472),
    "3:2": (1584, 1056),
    "2:3": (1056, 1584),
}

width, height = aspect_ratios["16:9"]

print(f"设置完成，将生成 {len(test_prompts)} 张测试图片")
print(f"图片尺寸: {width}x{height} (16:9)")

In [ ]:
# 使用多GPU生成图片
import time
from datetime import datetime

print("开始使用多GPU生成图片...")
print(f"开始时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*60)

generation_results = []
total_start_time = time.time()

# 依次生成5张图片
for i, test_case in enumerate(test_prompts, 1):
    print(f"\n正在生成第{i}张图片: {test_case['name']}")
    print(f"测试目标: {test_case['description']}")
    print(f"提示词: {test_case['prompt'][:100]}...")
    
    task_start_time = time.time()
    
    # 使用GPU管理器提交任务
    result = gpu_manager.submit_task(
        prompt=test_case['prompt'] + positive_magic["en"],
        negative_prompt=negative_prompt,
        seed=42 + i,
        width=width,
        height=height,
        guidance_scale=4.0,
        num_inference_steps=50,
        timeout=300
    )
    
    task_end_time = time.time()
    task_duration = task_end_time - task_start_time
    
    if result['success']:
        image = result['image']
        gpu_id = result['gpu_id']
        filename = f"{test_case['name']}_multigpu_test.png"
        image.save(filename)
        
        generation_results.append({
            'name': test_case['name'],
            'filename': filename,
            'gpu_id': gpu_id,
            'duration': task_duration,
            'success': True
        })
        
        print(f"✅ 图片已保存为: {filename}")
        print(f"   使用GPU: {gpu_id}")
        print(f"   生成时间: {task_duration:.2f}秒")
    else:
        error_msg = result.get('error', '未知错误')
        generation_results.append({
            'name': test_case['name'],
            'filename': None,
            'gpu_id': None,
            'duration': task_duration,
            'success': False,
            'error': error_msg
        })
        
        print(f"❌ 生成失败: {error_msg}")
        print(f"   尝试时间: {task_duration:.2f}秒")

total_end_time = time.time()
total_duration = total_end_time - total_start_time

print("\n" + "="*60)
print("所有图片生成完成！")
print(f"结束时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"总耗时: {total_duration:.2f}秒")

In [ ]:
# 生成结果汇总
print("\n📊 生成结果汇总:")
print("="*80)

successful_generations = [r for r in generation_results if r['success']]
failed_generations = [r for r in generation_results if not r['success']]

print(f"成功生成: {len(successful_generations)}/{len(generation_results)} 张图片")
print(f"失败数量: {len(failed_generations)} 张")

if successful_generations:
    avg_time = sum(r['duration'] for r in successful_generations) / len(successful_generations)
    print(f"平均生成时间: {avg_time:.2f}秒")
    
    # GPU使用统计
    gpu_usage = {}
    for r in successful_generations:
        gpu_id = r['gpu_id']
        if gpu_id not in gpu_usage:
            gpu_usage[gpu_id] = 0
        gpu_usage[gpu_id] += 1
    
    print("\nGPU使用分布:")
    for gpu_id, count in sorted(gpu_usage.items()):
        print(f"  GPU {gpu_id}: {count} 张图片")

print("\n📁 生成的文件:")
for r in generation_results:
    if r['success']:
        print(f"  ✅ {r['filename']} (GPU {r['gpu_id']}, {r['duration']:.1f}s)")
    else:
        print(f"  ❌ {r['name']} - 失败: {r.get('error', '未知错误')}")

if failed_generations:
    print("\n⚠️  失败的任务:")
    for r in failed_generations:
        print(f"  - {r['name']}: {r.get('error', '未知错误')}")

In [ ]:
# 清理资源
print("\n🧹 正在清理多GPU资源...")

try:
    gpu_manager.stop()
    print("✅ 多GPU管理器已停止")
    print("✅ 所有GPU工作进程已终止")
    print("✅ 资源清理完成")
except Exception as e:
    print(f"⚠️  清理过程中出现警告: {e}")

# 清理CUDA缓存
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ CUDA缓存已清理")

print("\n🎉 多GPU图像生成任务完成！")
print("\n💡 提示:")
print("  - 生成的图片保存在当前目录")
print("  - 文件名格式: {测试名称}_multigpu_test.png")
print("  - 如需重新运行，请重启内核以释放GPU内存")